# Relational graphe between people

## Get data

all imports

In [5]:
import data
import pandas as pd
import csv
import networkx as nx
import plotly.express as px
import datetime


get file

In [6]:
journal = "MALI_ER"

d = data.open_file(data.get_data(journal))



### Create the matrice

list of people

In [7]:
#need to work on this limite to find which one is good
DELETE_LIST = ["president","a","republique","ag","s","fall","february","ii","jean"]
LIMITE = 3
dictPers = {}
for i,pers  in enumerate(d["metadata-all"]["fr"]["all"]["per"]):
    
    # print(pers)
    value = d["metadata-all"]["fr"]["all"]["per"][pers]
    normalizePers = data.normalize_name(pers)
    # print(normalizePers)
    if normalizePers not in dictPers.keys():
        data.merge_same_name(dictPers,normalizePers,value,LIMITE)
    # else:
        # print(f"already in : {normalizePers}")
        
#delete manually wrong info
for delete_word in DELETE_LIST:
    if delete_word in dictPers.keys():
        dictPers.pop(delete_word)
    


before = len(d["metadata-all"]["fr"]["all"]["per"])
print(f"With limite {LIMITE} before: {before}")
# print(dictPers)
print(f"With limite {LIMITE} after: {len(dictPers)}")
print(dictPers)
dfDictPers = pd.DataFrame(list(dictPers.items()), columns=['Names', 'Values'])
dfDictPers.to_csv(f"../PeopleSaves/{journal}_dictPers.csv")


With limite 3 before: 185
With limite 3 after: 137
{'kabore': 2, 'ouattara': 3, 'macron': 19, 'kebbab': 6, 'shurkin': 2, 'barkhane': 17, 'lelievre': 3, 'barriere': 3, 'conrad': 2, 'soulages': 2, 'claudine': 3, 'sarkozy': 4, 'soral': 16, 'hollande': 9, 'poutine': 4, 'jinping': 2, 'valls': 4, 'trump': 12, 'erdogan': 12, 'djerma': 2, 'othman': 2, 'fodio': 2, 'ahmadou': 4, 'omar': 9, 'charlie': 3, 'cabu': 2, 'pelloux': 3, 'marianne': 5, 'charb': 2, 'bigot': 5, 'albaghdadi': 2, 'kadhafi': 18, 'washington': 7, 'philippe': 2, 'ibk': 2, 'bathily': 7, 'coulibaly': 4, 'lassana': 2, 'kacimi': 21, 'foucher': 6, 'gueant': 3, 'capital': 2, 'couriol': 4, 'mitterrand': 3, 'schaffhauser': 2, 'dumas': 2, 'cartier': 2, 'lugan': 4, 'alqaeda': 2, 'bouhafs': 5, 'michel': 4, 'attali': 2, 'keita': 5, 'ministre': 4, 'cisse': 2, 'ghali': 31, 'dicko': 2, 'assa': 7, 'traore': 72, 'adama': 8, 'alsaharaoui': 2, 'koufa': 4, 'droukdal': 6, 'zemmour': 3, 'kelly': 3, 'dieudonne': 2, 'colbert': 2, 'floyd': 4, 'ferguson'

matrice

In [8]:
SEUIL = 4
print(f"Taille de la liste avant la suppresion des personnes à moins de {SEUIL} liens : {len(dictPers)}")
cleanList = [key for key, value in dictPers.items() if value >= SEUIL]
print(f"Taille après : {len(cleanList)}")
matrice = pd.DataFrame(0, index=cleanList, columns=cleanList)
# print(matrice)
matrice.to_csv(f"../PeopleSaves/{journal}_Matrice.csv")

Taille de la liste avant la suppresion des personnes à moins de 4 liens : 137
Taille après : 57


### Fill the matrice

get all the per dict from the journal

In [9]:
def dict_get_bis(journal,key):
    ret = []
    arts = data.open_file(data.get_data(journal))["data-all"]
    # print(arts)
    for year in arts:
        # print(arts[year])
        for month in arts[year]:
            # print(arts[year][month])
            for day in arts[year][month]:
                # print(arts[year][month][day])
                for art,i in enumerate(arts[year][month][day]):
                    ret.append(arts[year][month][day][art][key])
    return ret
                    
    
allPeopleLists = dict_get_bis(journal,"per")
allPeopleDict = {key:v for key,v in enumerate(allPeopleLists)}
print(f"ICI : {allPeopleDict}")
print(d["metadata-all"]["fr"]["all"]["num"])
print(len(allPeopleDict))
allPeopleDf = pd.DataFrame.from_dict(allPeopleDict,orient="index")
# print(allPeopleDf)
allPeopleDf.to_csv(f"../PeopleSaves/{journal}_allPeopleDf.csv")    

ICI : {0: {'Roch Kaboré': 2}, 1: {}, 2: {'Alassane Ouattara': 3, 'Emmanuel Macron': 2}, 3: {'Linda Kebbab': 6}, 4: {'Michael Shurkin': 2, 'Barkhane': 2}, 5: {}, 6: {'Barkhane': 2}, 7: {}, 8: {'Lelièvre': 3, 'Alain Barrière': 3, 'Nick Conrad': 2, 'Pierre Soulages': 2, 'Claudine': 3}, 9: {'Macron': 3}, 10: {'Macron': 2, 'Barkhane': 2}, 11: {'Nicolas Sarkozy': 2, 'Sarkozy': 4, 'Alain Soral': 3, 'François Hollande': 2, 'Vladimir Poutine': 2, 'Xi Jinping': 2, 'Valls': 3, 'Manuel Valls': 2, 'Donald Trump': 4, 'Trump': 4, 'Erdoğan': 2, 'Président': 2, 'Macron': 2}, 12: {}, 13: {'Djerma': 2, 'Othman': 2, 'dan Fodio': 2, 'Seku Ahmadou': 2, 'El Hadj Omar': 2}, 14: {'Charlie': 3, 'Cabu': 2, 'Pelloux': 3, 'Marianne': 2, 'Charb': 2}, 15: {'Barkhane': 2}, 16: {}, 17: {'Emmanuel Macron': 4, 'Guillaume Bigot': 5}, 18: {'Abou Bakr al-Baghdadi': 2, 'Kadhafi': 2, 'Washington': 2}, 19: {'El Hadj Omar': 5, 'Édouard Philippe': 2, 'Seku Ahmadou': 2}, 20: {}, 21: {'Barkhane': 2, 'IBK': 2}, 22: {'Lassana Bathi

get the matrice and fill it 

In [10]:
matriceFromCsv = pd.read_csv(f"../PeopleSaves/{journal}_Matrice.csv")
clearList = matriceFromCsv.columns[1:]
indexDict = {key: value for value,key in enumerate(matriceFromCsv[cleanList])}
reverseDict = {value: key for key,value in indexDict.items()}
# print(f"Index : {indexDict}")
allPeopleDfFromCsv = pd.read_csv(f"../PeopleSaves/{journal}_AllPeopleDf.csv")
allPeopleDictFromCsv = allPeopleDfFromCsv.to_dict(orient="index")
cpt = 0


#Create a set of normalized person names for faster lookup
for article in allPeopleDictFromCsv.values():
    print(f"{cpt/len(allPeopleDictFromCsv)*100}%")
    normalized_names = {data.normalize_name(name) for name,value in article.items() if value>0 and name != "Unnamed: 0"}
    # print(clearList)
    for pers in normalized_names:
        others = [personne for personne in normalized_names if personne != pers]
        # print(f"La personne est  :  {pers}, les autres sont : {others}")
        for other in others:
            closePers = data.get_merge_name(clearList, pers)
            # print(f"La personne de la liste clearList se rapprochant le plus de {pers} est : {closePers}")
            closeOther = data.get_merge_name(clearList, other)
            # print(f"La personne de la liste clearList se rapprochant le plus de {other} est : {closeOther}")
            # print(f"La valeur de la matrice se trouvant à l'intersection {matriceFromCsv.at[indexDict[closePers],closeOther]}")
            matriceFromCsv.at[indexDict[closePers],closeOther] += 1
    cpt +=1
    
matriceFromCsv.to_csv(f"../PeopleSaves/{journal}_FinalMatrice.csv")








0.0%
1.4705882352941175%
2.941176470588235%
4.411764705882353%
5.88235294117647%
7.352941176470589%
8.823529411764707%
10.294117647058822%
11.76470588235294%
13.23529411764706%
14.705882352941178%
16.176470588235293%
17.647058823529413%
19.11764705882353%
20.588235294117645%
22.058823529411764%
23.52941176470588%
25.0%
26.47058823529412%
27.941176470588236%
29.411764705882355%
30.88235294117647%
32.35294117647059%
33.82352941176471%
35.294117647058826%
36.76470588235294%
38.23529411764706%
39.705882352941174%
41.17647058823529%
42.64705882352941%
44.11764705882353%
45.588235294117645%
47.05882352941176%
48.529411764705884%
50.0%
51.470588235294116%
52.94117647058824%
54.41176470588235%
55.88235294117647%
57.35294117647059%
58.82352941176471%
60.29411764705882%
61.76470588235294%
63.23529411764706%
64.70588235294117%
66.17647058823529%
67.64705882352942%
69.11764705882352%
70.58823529411765%
72.05882352941177%
73.52941176470588%
75.0%
76.47058823529412%
77.94117647058823%
79.41176470588

test heatmap

In [11]:
finalOne = pd.read_csv(f"../PeopleSaves/{journal}_FinalMatrice.csv",index_col=0)
finalOne.index = finalOne.index.map(reverseDict)
fig = px.imshow(finalOne.values,
                labels=dict(x="Columns", y="Rows", color="Distance"),
                x=finalOne.columns,
                y=finalOne.index,
                color_continuous_scale='viridis')

fig.update_layout(title_text='Heatmap of the Matrix', xaxis_nticks=len(finalOne.columns),yaxis_nticks=len(finalOne.columns))

fig.show()

Transformation en graphe

In [12]:
finalOne = pd.read_csv(f"../PeopleSaves/{journal}_FinalMatrice.csv", index_col=1)
# print(finalOne)
G = nx.from_pandas_adjacency(finalOne)



nx.write_gml(G,f"../PeopleSaves/{journal}_graphe.gml") 

# With only the ten most important person

## get the top 10 

In [18]:
dfDictPers = pd.read_csv(f"../PeopleSaves/{journal}_DictPers.csv")
dictPers = dfDictPers.set_index('Names').to_dict()["Values"]
print(dictPers)
top10_dict = {}
for name,value in dictPers.items():

    if len(top10_dict)<10:
        top10_dict[name] = value
    else:
        min_key = min(top10_dict, key=top10_dict.get)
        min_value = min(top10_dict.values())
        if value > min_value:
            del top10_dict[min_key]
            top10_dict[name] = value
    
names = list(top10_dict.keys())
values = list(top10_dict.values())
top10_df = pd.DataFrame({'Names':names,'Values': values})  

print(top10_df["Names"])
print(list(top10_df["Names"]))

{'kabore': 2, 'ouattara': 3, 'macron': 19, 'kebbab': 6, 'shurkin': 2, 'barkhane': 17, 'lelievre': 3, 'barriere': 3, 'conrad': 2, 'soulages': 2, 'claudine': 3, 'sarkozy': 4, 'soral': 16, 'hollande': 9, 'poutine': 4, 'jinping': 2, 'valls': 4, 'trump': 12, 'erdogan': 12, 'djerma': 2, 'othman': 2, 'fodio': 2, 'ahmadou': 4, 'omar': 9, 'charlie': 3, 'cabu': 2, 'pelloux': 3, 'marianne': 5, 'charb': 2, 'bigot': 5, 'albaghdadi': 2, 'kadhafi': 18, 'washington': 7, 'philippe': 2, 'ibk': 2, 'bathily': 7, 'coulibaly': 4, 'lassana': 2, 'kacimi': 21, 'foucher': 6, 'gueant': 3, 'capital': 2, 'couriol': 4, 'mitterrand': 3, 'schaffhauser': 2, 'dumas': 2, 'cartier': 2, 'lugan': 4, 'alqaeda': 2, 'bouhafs': 5, 'michel': 4, 'attali': 2, 'keita': 5, 'ministre': 4, 'cisse': 2, 'ghali': 31, 'dicko': 2, 'assa': 7, 'traore': 72, 'adama': 8, 'alsaharaoui': 2, 'koufa': 4, 'droukdal': 6, 'zemmour': 3, 'kelly': 3, 'dieudonne': 2, 'colbert': 2, 'floyd': 4, 'ferguson': 2, 'louvet': 2, 'joly': 2, 'louis': 6, 'abdelmale

## Create the empty matrice

In [14]:
print(names)
print(cleanList)

matrix_df  = pd.DataFrame(0,index = names, columns = cleanList)
print(matrix_df)
matrix_df.to_csv(f"../PeopleSaves/{journal}_Top10_Matrice.csv")

['macron', 'barkhane', 'soral', 'trump', 'erdogan', 'kadhafi', 'kacimi', 'ghali', 'traore', 'habib']
['macron', 'kebbab', 'barkhane', 'sarkozy', 'soral', 'hollande', 'poutine', 'valls', 'trump', 'erdogan', 'ahmadou', 'omar', 'marianne', 'bigot', 'kadhafi', 'washington', 'bathily', 'coulibaly', 'kacimi', 'foucher', 'couriol', 'lugan', 'bouhafs', 'michel', 'keita', 'ministre', 'ghali', 'assa', 'traore', 'adama', 'koufa', 'droukdal', 'floyd', 'louis', 'brakni', 'emie', 'beghal', 'lecointre', 'huyghe', 'hitler', 'kersten', 'francis', 'habib', 'ihuellou', 'fenech', 'zuckerberg', 'wagner', 'maiga', 'berry', 'nuland', 'ryabkov', 'wyke', 'sinclair', 'fleck', 'cattigan', 'itno', 'deby']
          macron  kebbab  barkhane  sarkozy  soral  hollande  poutine  valls  \
macron         0       0         0        0      0         0        0      0   
barkhane       0       0         0        0      0         0        0      0   
soral          0       0         0        0      0         0        0    

## fill the matrice

In [15]:
matriceTop10FromCsv = pd.read_csv(f"../PeopleSaves/{journal}_Top10_Matrice.csv",index_col=0)
clearList = matriceFromCsv.columns[1:]
indexDict = {key: value for value,key in enumerate(matriceFromCsv[cleanList])}
# reverseDict = {value: key for key,value in indexDict.items()}
allPeopleDfFromCsv = pd.read_csv(f"../PeopleSaves/{journal}_AllPeopleDf.csv")
allPeopleDictFromCsv = allPeopleDfFromCsv.to_dict(orient="index")
cpt = 0

print(matriceTop10FromCsv)
for article in allPeopleDictFromCsv.values():
    print(f"{cpt/len(allPeopleDictFromCsv)*100}%")
    normalized_names = {data.normalize_name(name) for name,value in article.items() if value>0 and name != "Unnamed: 0"}
    # print(clearList)
    for pers in normalized_names:
        # print(f"La personne est  :  {pers}, les autres sont : {others}")
        closePers = data.get_merge_name(clearList, pers)
        print(f"Pers : {closePers}")
        for name in names:
            if name in normalized_names:
                closename = data.get_merge_name(clearList, name)
                print(f"top10:  {name}")
                # print(f"La personne de la liste clearList se rapprochant le plus de {pers} est : {closePers}")
                # print(f"La personne de la liste clearList se rapprochant le plus de {other} est : {closeOther}")
                # print(f"La valeur de la matrice se trouvant à l'intersection {matriceFromCsv.at[indexDict[closePers],closeOther]}")
                print(closename)
                matriceTop10FromCsv.loc[closename,closePers] += 1
    cpt +=1
    
matriceTop10FromCsv.to_csv(f"../PeopleSaves/{journal}_Top10_FinalMatrice.csv")

          macron  kebbab  barkhane  sarkozy  soral  hollande  poutine  valls  \
macron         0       0         0        0      0         0        0      0   
barkhane       0       0         0        0      0         0        0      0   
soral          0       0         0        0      0         0        0      0   
trump          0       0         0        0      0         0        0      0   
erdogan        0       0         0        0      0         0        0      0   
kadhafi        0       0         0        0      0         0        0      0   
kacimi         0       0         0        0      0         0        0      0   
ghali          0       0         0        0      0         0        0      0   
traore         0       0         0        0      0         0        0      0   
habib          0       0         0        0      0         0        0      0   

          trump  erdogan  ...  maiga  berry  nuland  ryabkov  wyke  sinclair  \
macron        0        0  ...      0   

## Into graph

In [16]:
finalOneTop10 = pd.read_csv(f"../PeopleSaves/{journal}_Top10_FinalMatrice.csv", index_col=0)
# print(finalOne)
GTop10 = nx.DiGraph()

# Ajouter les nœuds correspondant aux colonnes du DataFrame
for column in matrix_df.columns:
    GTop10.add_node(column)

# Ajouter les arcs pondérés (edges) au graphe en utilisant les valeurs de la matrice
for i in range(len(matrix_df)):
    for j in range(len(matrix_df.columns) - 1):
        if i != j:  # Éviter de créer des arcs du nœud à lui-même
            weight = matrix_df.iloc[i, j]  # Utiliser la valeur de la matrice comme poids
            GTop10.add_edge(
                matrix_df.columns[i],
                matrix_df.columns[j],
                weight=str(weight)  # Convertir le poids en chaîne de caractères
            )

nx.write_gml(GTop10, f"../PeopleSaves/{journal}_Top10_graphe.gml")